In [1]:
import pandas as pd
import openap

In [2]:
actype = "A320"
mass_takeoff = 66300

fuelflow = openap.FuelFlow(actype)


In [3]:
df = pd.read_csv("data/flight_a319_opensky.csv", parse_dates=["timestamp"])

df

,timestamp,icao24,typecode,callsign,origin,destination,latitude,longitude,altitude,groundspeed,track,vertical_rate
0,2018-01-02 19:53:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,49.085861,2.349666,8200.0,255.0,327.804266,1920.0
1,2018-01-02 19:54:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,49.153427,2.310861,9475.0,290.0,9.722018,960.0
2,2018-01-02 19:55:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,49.233367,2.361121,9975.0,332.0,23.243919,640.0
3,2018-01-02 19:56:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,49.319916,2.418471,11225.0,353.0,23.517962,2752.0
4,2018-01-02 19:57:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,49.411652,2.478896,14175.0,368.0,23.566915,3584.0
...,...,...,...,...,...,...,...,...,...,...,...,...
111,2018-01-02 21:44:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,58.978659,17.706646,13250.0,343.0,24.471621,-1408.0
112,2018-01-02 21:45:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,59.062826,17.781088,11800.0,326.0,24.443955,-1344.0
113,2018-01-02 21:46:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,59.142609,17.852051,10400.0,307.0,24.397686,-1216.0
114,2018-01-02 21:47:00+00:00,3.946000e+12,a319,AFR91HL,LBG,BMA,59.217286,17.918701,9125.0,291.0,24.541618,-1216.0


In [4]:
df = df.assign(d_ts=lambda d: d.timestamp.diff().dt.total_seconds().bfill())


In [5]:
mass_current = mass_takeoff

fuel_every_step = []
for i, row in df.iterrows():
    ff = fuelflow.enroute(
        mass=mass_current, tas=row.groundspeed, alt=row.altitude, vs=row.vertical_rate
    )
    fuel = ff * row.d_ts
    fuel_every_step.append(ff * row.d_ts)
    mass_current -= fuel

df = df.assign(fuel=fuel_every_step)

In [6]:
total_fuel = df.fuel.sum()
print(f"Total fuel: {total_fuel:.2f} kg")

Total fuel: 5543.12 kg
